In [3]:
import sys

print(f"🐍 Python executando em: {sys.executable}")
print(f"ℹ️ Versão do Python: {sys.version.split()[0]}\n")

libs = {
    "opencv-python": "cv2",
    "mediapipe": "mediapipe",
    "tensorflow": "tensorflow",
    "numpy": "numpy"
}

print("=== CHECAGEM DE VERSÕES ===")
for nome_pip, nome_import in libs.items():
    try:
        mod = __import__(nome_import)
        # Alguns módulos usam __version__, outros podem variar, mas o padrão é esse
        version = getattr(mod, "__version__", "Versão não encontrada")
        print(f"✅ {nome_pip}: {version}")
    except ImportError:
        print(f"❌ {nome_pip}: NÃO INSTALADO (ou não visível para este Kernel)")

🐍 Python executando em: C:\Users\scien\.conda\envs\faceproj\python.exe
ℹ️ Versão do Python: 3.10.19

=== CHECAGEM DE VERSÕES ===
✅ opencv-python: 4.11.0
✅ mediapipe: 0.10.14
✅ tensorflow: 2.16.1
✅ numpy: 1.26.4


In [5]:
import os
import cv2
import time
import mediapipe as mp
import numpy as np

# Configurações Globais
RAW_DATA_DIR = os.path.join("data", "raw")
IMG_SIZE = 160 # Tamanho padrão para o modelo

# Cria a pasta principal se não existir
os.makedirs(RAW_DATA_DIR, exist_ok=True)

def coletar_dados(nome_pessoa, qtd_maxima=50):
    save_path = os.path.join(RAW_DATA_DIR, nome_pessoa)
    os.makedirs(save_path, exist_ok=True)
    
    cap = cv2.VideoCapture(0)
    mp_face = mp.solutions.face_detection
    
    print(f"📸 Coletando dados para: {nome_pessoa.upper()}")
    print("Tecle [S] para Salvar | [Q] para Sair")
    
    count = len(os.listdir(save_path))
    
    with mp_face.FaceDetection(min_detection_confidence=0.6) as face:
        while cap.isOpened():
            ok, frame = cap.read()
            if not ok: break
            
            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face.process(rgb)
            
            face_crop = None
            
            if results.detections:
                # Pega a maior detecção (caso tenha mais de uma pessoa)
                detection = max(results.detections, key=lambda d: d.score[0])
                bbox = detection.location_data.relative_bounding_box
                
                h, w, _ = frame.shape
                x, y = int(bbox.xmin * w), int(bbox.ymin * h)
                w_box, h_box = int(bbox.width * w), int(bbox.height * h)
                
                # Desenha o quadrado
                cv2.rectangle(frame, (x, y), (x + w_box, y + h_box), (0,255,0), 2)
                
                # Recorte seguro (evita erro se sair da tela)
                face_crop = frame[max(0, y):min(h, y+h_box), max(0, x):min(w, x+w_box)]
            
            cv2.putText(frame, f"Salvas: {count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow(f"Coleta - {nome_pessoa}", frame)
            
            k = cv2.waitKey(1)
            if k == ord('s') or k == ord('S'):
                if face_crop is not None and face_crop.size > 0:
                    face_resized = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE))
                    file_name = f"{nome_pessoa}_{int(time.time()*1000)}.jpg"
                    cv2.imwrite(os.path.join(save_path, file_name), face_resized)
                    count += 1
                    print(f"✅ Foto salva! Total: {count}")
                else:
                    print("⚠️ Rosto não detectado!")
            
            if k == ord('q') or k == ord('Q') or count >= qtd_maxima:
                break
                
    cap.release()
    cv2.destroyAllWindows()
    print("✅ Coleta finalizada.")

# --- EXECUÇÃO ---
# Mude o nome aqui e rode a célula (ex: "alex", depois mude para "fabiana" e rode de novo)
coletar_dados("fabiana", qtd_maxima=50)

📸 Coletando dados para: FABIANA
Tecle [S] para Salvar | [Q] para Sair
✅ Foto salva! Total: 1
✅ Foto salva! Total: 2
✅ Foto salva! Total: 3
✅ Foto salva! Total: 4
✅ Foto salva! Total: 5
✅ Foto salva! Total: 6
✅ Foto salva! Total: 7
✅ Foto salva! Total: 8
✅ Foto salva! Total: 9
✅ Foto salva! Total: 10
✅ Foto salva! Total: 11
✅ Foto salva! Total: 12
✅ Foto salva! Total: 13
✅ Foto salva! Total: 14
✅ Foto salva! Total: 15
✅ Foto salva! Total: 16
✅ Foto salva! Total: 17
✅ Foto salva! Total: 18
✅ Foto salva! Total: 19
✅ Foto salva! Total: 20
✅ Foto salva! Total: 21
✅ Foto salva! Total: 22
✅ Foto salva! Total: 23
✅ Foto salva! Total: 24
✅ Foto salva! Total: 25
✅ Foto salva! Total: 26
✅ Foto salva! Total: 27
✅ Foto salva! Total: 28
✅ Foto salva! Total: 29
✅ Foto salva! Total: 30
✅ Foto salva! Total: 31
✅ Foto salva! Total: 32
✅ Foto salva! Total: 33
✅ Foto salva! Total: 34
✅ Foto salva! Total: 35
✅ Foto salva! Total: 36
✅ Foto salva! Total: 37
✅ Foto salva! Total: 38
✅ Foto salva! Total: 39
✅ F

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Configurações de Treino
BATCH_SIZE = 16
IMG_SHAPE = (160, 160)
DATA_DIR = os.path.join("data", "raw") # Pega direto da pasta onde salvamos

# 1. Carregamento Inteligente (Já divide Treino/Validação e Redimensiona)
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SHAPE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SHAPE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"\n🤖 Classes encontradas: {class_names}")

# 2. Performance (Cache na memória para ser rápido)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 3. Data Augmentation (Cria variações das fotos para o modelo aprender melhor)
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# 4. Construção do Modelo
base_model = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

# CORREÇÃO DO BUG: O preprocessamento fica DENTRO do modelo
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x) # <--- AQUI A MÁGICA: Converte 0-255 para -1 a 1 automaticamente
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5. Treinar
history = model.fit(train_ds, validation_data=val_ds, epochs=15)

# Salvar
model.save("face_classifier.keras")
print("✅ Modelo salvo com sucesso!")

Found 100 files belonging to 2 classes.
Using 80 files for training.
Found 100 files belonging to 2 classes.
Using 20 files for validation.

🤖 Classes encontradas: ['alex', 'fabiana']
Epoch 1/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 643ms/step - accuracy: 0.5500 - loss: 0.9319 - val_accuracy: 0.7000 - val_loss: 0.5238
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.7125 - loss: 0.6422 - val_accuracy: 0.9000 - val_loss: 0.3748
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.6625 - loss: 0.6911 - val_accuracy: 0.9000 - val_loss: 0.3086
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.8250 - loss: 0.3914 - val_accuracy: 0.9500 - val_loss: 0.2051
Epoch 5/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.9000 - loss: 0.3213 - val_accuracy: 0.9500 - val_loss: 0.1775
Epoch 6/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.8625 - loss: 0.3069 - val_accuracy: 0.9500 - val_loss: 0.1941
Epoch 7/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accurac

In [7]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

# Carrega o modelo
model = tf.keras.models.load_model("face_classifier.keras")
class_names = ["alex", "fabiana"] # Importante: Ordem alfabética (padrão do diretório)

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_detection

print("🎥 Iniciando reconhecimento... [Q] para sair")

with mp_face.FaceDetection(min_detection_confidence=0.6) as face:
    while cap.isOpened():
        ok, frame = cap.read()
        if not ok: break
        
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face.process(rgb)
        
        if results.detections:
            for detection in results.detections:
                # 1. Recorte do Rosto
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x, y = int(bbox.xmin * w), int(bbox.ymin * h)
                w_box, h_box = int(bbox.width * w), int(bbox.height * h)
                
                # Expande um pouquinho a caixa para pegar o rosto todo
                face_crop = frame[max(0, y):min(h, y+h_box), max(0, x):min(w, x+w_box)]
                
                if face_crop.size > 0:
                    # 2. Prepara imagem para o modelo
                    img = cv2.resize(face_crop, (160, 160))
                    img_array = np.expand_dims(img, axis=0) # Cria o lote de 1 imagem
                    
                    # 3. Predição (O modelo já faz a normalização internamente!)
                    prediction = model.predict(img_array, verbose=0)
                    index = np.argmax(prediction)
                    confidence = prediction[0][index]
                    
                    # 4. Resultado
                    nome = class_names[index].upper()
                    cor = (0, 255, 0) if confidence > 0.8 else (0, 0, 255)
                    texto = f"{nome} ({confidence:.0%})"
                    
                    cv2.rectangle(frame, (x, y), (x+w_box, y+h_box), cor, 2)
                    cv2.putText(frame, texto, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, cor, 2)
        
        cv2.imshow("Reconhecimento Facial", frame)
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

🎥 Iniciando reconhecimento... [Q] para sair
